In [266]:
# Import numpy, Enum and Queue
import numpy as np
from enum import Enum
from queue import Queue
from math import sqrt
from math import inf

In [267]:
# Define a start and goal location
s = (0, 0)
g = (4, 4)
found_path = False
# Define your grid-based state space of obstacles and free space
grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [268]:
print(np.shape(grid)[0],np.shape(grid)[1])


5 6


In [269]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    LEFTUP = (-1, -1, sqrt(2))
    LEFTDOWN = (1, -1, sqrt(2))
    RIGHTUP = (-1, 1, sqrt(2))
    RIGHTDOWN = (1, 1, sqrt(2))

    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])
            
    
def valid_actions(grid, current_node):
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN, Action.LEFTUP, Action.LEFTDOWN, Action.RIGHTUP, Action.RIGHTDOWN]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = (current_node)
    
    
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
    if (x - 1 < 0 or y - 1 < 0) or grid[x-1, y-1] == 1:
        valid.remove(Action.LEFTUP)
    if (x + 1 > n or y - 1 < 0) or grid[x+1, y-1] == 1:
        valid.remove(Action.LEFTDOWN)
    if (x - 1 < 0 or y + 1 > m) or grid[x-1, y+1] == 1:
        valid.remove(Action.RIGHTUP)
    if (x + 1 > n or y + 1 > m) or grid[x+1, y+1] == 1:
        valid.remove(Action.RIGHTDOWN)

    return valid

In [280]:
def breadth_first(grid, start, goal):
    q = Queue()
    q.put((0, start))
    visited = set(start)
    branch = {}
    found = False
    width = np.shape(grid)[1]
    mapa = np.zeros(np.shape(grid)[0]-1 + (np.shape(grid)[1]-1) * width + 1)
    mapa[mapa > -1] = inf 
    print(len(mapa))
    while not q.empty(): 
        item = q.get()
        current_node = item[1]
        
        if current_node == start:
            current_cost = 0.0
        else:              
            current_cost = branch[current_node][0]
        
        if current_node == goal: 
            print('Found a path.')
            found = True
            break
        else:
            valid = valid_actions(grid, current_node)
            for action in valid:
                da = action.value
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = action.cost + current_cost
                mapa[current_node[0] + current_node[1] * width] = current_cost
                if next_node not in visited:
                    visited.add(next_node)
                    q.put((branch_cost, next_node))    
                    branch[next_node] = (branch_cost, current_node, action)
   
    print(current_cost)
    return mapa

In [281]:
mapa = breadth_first(grid, s, g)
#print(path)
#print(steps)
print(mapa)
print(grid)


35
Found a path.
10.485281374238571
[ 0.          1.          2.          3.          4.                 inf
         inf         inf         inf  3.41421356  4.41421356         inf
  6.82842712  5.82842712  4.82842712  4.41421356  4.82842712         inf
  7.24264069  6.24264069         inf         inf         inf         inf
  7.65685425  7.24264069  7.65685425         inf         inf         inf
  8.65685425  8.24264069  8.65685425  9.07106781 10.07106781]
[[0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 1 0 0]
 [0 0 0 1 1 0]
 [0 0 0 1 0 0]]
